### Simulation des rapports des visites bimestrielles et des interventions de maintenance annuelle des issues et des niches 

* Définition du calendier et des numéros d'OT (fermeturesJours.csv)
* Production des rapports (absences aléatoires)
* Upload des rapports sur le serveur

Cette simulation permet de tester les choix qui sont faits sur la forme des rapports.  
la simulation fournit un jeu de données pour développer les outils de traitements des données pour la visualisation des résultats.



In [ ]:
#! pip install -r requirements.txt
import pandas as pd, numpy as np
import glob, re,json,io,os
from datetime import datetime, timedelta
from time import sleep
import plotly.graph_objects as go
pd.options.display.max_colwidth = 100
from numpy.random import random
from numpy.random import choice
rng = np.random.default_rng()
from google.cloud import bigquery,storage
project_id = 'dett-stt'
clientB = bigquery.Client(project_id)
storageC=storage.Client(project_id)

Index(['CodeEx', 'PC', 'Tatouage', 'triCode', 'Fermeture', 'OQ'], dtype='object')

#### Programmation des dates de fermetures et simulation des OT

In [ ]:

issues=pd.read_csv('https://raw.githubusercontent.com/ExploitIdF/Referentiel_Tunnels/refs/heads/main/issuesFermetures.csv')#[[  'Tatouage',   'Fermeture']]
# ['CodeEx', 'PC', 'Tatouage', 'triCode', 'Fermeture']
issues['OQ']=range(290)
fermetures=issues['Fermeture'].value_counts()
lenFer=len(fermetures )
if False:
  fermJours=[]
  for k in range(6):
    jfTmp=pd.Series([(5+k*8)*7+ int(random()*4)*7+int(random()*4) for i in range(lenFer)],index=fermetures.index,name='jour')
    otTmp=pd.Series([456000+i*1000 for i in range(lenFer)],index=fermetures.index,name='OT')
    ordTmp=pd.Series( k,index=fermetures.index,name='ordreVB')
    fermJours.append(pd.concat([jfTmp,otTmp,ordTmp],axis=1))
  fermJours=pd.concat(fermJours).reset_index()
  fermJours.to_csv('fermeturesJours.csv',index=False)

  fermeturesJours=pd.read_csv('fermeturesJours.csv')
  issJours=issues.merge(fermeturesJours,on='Fermeture', how='outer')
  issJours['OT']=issJours['OT']+issJours['OQ']+100
  #issJours=issJours[['Tatouage', 'Fermeture', 'jour', 'OT']]

  minutes=pd.Series([int(random()*240) for i in range(len(issJours))],index=issJours.index,name='minute')

  VisiteIssues=pd.concat([issJours,minutes],axis=1)
  VisiteIssues['HoroDate']=pd.to_datetime("24/"+VisiteIssues['jour'].astype(str)+" 23",format='%y/%j %H')+pd.to_timedelta(VisiteIssues['minute']*60000000000)

  VisiteIssues[['Fermeture','ordreVB', 'OT','CodeEx', 'Tatouage', 'HoroDate']].to_csv('ProgrVisiteIssues.csv',index=False)

In [68]:
fermJours

,Fermeture,jour,OT,ordreVB
0,A14&NEU-W,42,456000,0
1,A14&NEU-Y,44,457000,0
2,BLN&CRN-I,35,458000,0
3,BLN&CRN-E,43,459000,0
4,BAP&SCL-W,36,460000,0
...,...,...,...,...
175,BOI-Y,338,481000,5
176,FLF-W,322,482000,5
177,FLF-Y,318,483000,5
178,ORL,330,484000,5


In [89]:
# Lecture du tableau des points de contrôle initial. Ce format devrait changer ...
vbIa=pd.read_csv('../_static/controleVB_IA.csv',encoding='UTF-8').iloc[:,1:]
lstPC=vbIa['PointControle'].value_counts().reset_index()
lstPC['ordrePC']=lstPC['PointControle'].str.split('.').apply(lambda x: x[0]).astype(int).astype(str).str.zfill(2) 
lstPC=lstPC.sort_values('ordrePC').reset_index(drop=True) #.set_index('ordrePC',drop=True)
lstLst=[]
for i in range(len(lstPC)) :
  vbIi=vbIa[vbIa['PointControle']==lstPC.loc[i,'PointControle']]
  lstCom=[]
  for j in vbIi.index:
    lstCom.append(vbIi.loc[j,'Note'][0]+'_'+vbIi.loc[j,'ResulControle'])
  lstLst.append(sorted(lstCom, reverse=False))

lstNbrResul=[len(ll) for ll in lstLst]
lsP=[[round(.2/(i-1),3)]*(i-1) +[1-(i-1)*round(.2/(i-1),3)]    for i in lstNbrResul]
lstLstR=[['R'+str(i) for i in range(j)] for j in lstNbrResul ]

tabRC=[]
for i in range(len(lstPC)) :
  for j in range(lstNbrResul[i]) :
    tabRC.append(['P'+ lstPC.loc[i,'ordrePC'], lstPC.loc[i,'PointControle'],lstLstR[i][j],lstLst[i][j],lsP[i][j]  ] )
pd.DataFrame(tabRC,columns=['codePC','PC','codeRC', 'RC', 'Probab']).to_csv('controleVB_IS.csv',index=False)
# Dans le futur, on devrait entrer directement une table de ce type.


## Création des rapports
On définit une fonction qui pour une fermeture programmée un jour donné (correspondant à un OT père) enregistre les rapports des issues de la fermeture, avec des omissions aléatoires.  
Ensuite, on applique la fonction à toutes les fermetures programmées, avec des omissions aléatoires.  




In [95]:
# fonction qui, pour toutes les issues d'une fermeture et pour une nuit, fabrique les fichiers des rapports VBIS et l'enregistre dans le répertoire 24-3
# Appelée dans la cellule suivante

controleVB_IS=pd.read_csv('controleVB_IS.csv')
VisiteIssues=pd.read_csv('ProgrVisiteIssues.csv')
VisiteIssues['HoroDate']=pd.to_datetime( VisiteIssues['HoroDate'],format='%Y-%m-%d %H:%M:%S')
fermJours=0 #pd.read_csv('fermeturesJours.csv')
ComAls=[x for x in  ['Commentaire Aléatoire0_Très','Commentaire Aléatoire1_îàèù','Commentaire Aléatoire2_Trop Û','Commentaire Aléatoire3_Trois_Ècî',
                     'Commentaire Aléatoire4_Qûàtré']]

def simuRapp(ferm,ord):
    vst=VisiteIssues[(VisiteIssues['Fermeture']==ferm)&(VisiteIssues['ordreVB']==ord)].reset_index(drop=True)  
    for k in range(len(vst)):
        if random()<.95:
            dateFichier= str(int((vst.loc[k,'HoroDate']-datetime(2024,1,1)).total_seconds()+1000*random()))
            flNm='24-3/ISVB-'+ str(vst.loc[k,'OT'])+'-'+dateFichier+ '.json'
            flSr='{"Tatouage":"'+str(VisiteIssues.loc[k,'Tatouage'])+'",'
            flSr=flSr+'"HoroDate":"'+str(VisiteIssues.loc[k,'HoroDate'])+'",'
            flSr=flSr+'"Agent":"'+choice(['Karl','Karen', 'Kim','Kamel','Kun']) +'"'
            for n in range(len(controleVB_IS['codePC'].unique())):
                cn=controleVB_IS[controleVB_IS['codePC']== controleVB_IS['codePC'].unique()[n] ][['codeRC','Probab']]
                flSr=flSr+',"PC'+str(n)+'":"'+choice(a=list(cn['codeRC']),p=list(cn['Probab']))+'"'+' ,"CM'+str(n)+'":"'+choice(a=ComAls)+'"'            
            flSr=flSr+'}'
            with open(flNm,'bw') as fl:
                fl.write(flSr.encode('UTF8'))

# Création des rapports par application de la fonction avec une probabilité d'omission
repertoire ='24-3/'
if True :
    filenames = next(os.walk(repertoire), (None, None, []))[2]
    for fl in filenames:
        os.remove(repertoire + fl)

    for ferm in list(VisiteIssues['Fermeture'].unique()):
        for j in range(6):
            if random()<0.5:
                simuRapp(ferm,j)
filenames = next(os.walk(repertoire), (None, None, []))[2]



## Upload des rapports sur le serveur Google Storage
On vide le répertoire cible (Storage : issues-secours/rapports-visites/) et  on charge les fichiers avec une pause de 3 secondes 
pour tenir compte de l'import dans BQ par la fonction qui est déclenchée par le chargement.



In [96]:
# On vide le répertoire cible (Storage : issues-secours/rapports-visites/) et on recharge les fichiers avec une pause de 3 seconde pour tenir compte de l'import dans BQ
bucket = storageC.get_bucket('issues-secours')
blobs = storageC.list_blobs(bucket)
for blob in blobs:
    if 'ISV' in blob.name:
        blob.delete()

for fl in filenames[:]:
    blob = bucket.blob('rapports-visites/'+fl)
    blob.upload_from_filename('24-3/'+ fl, if_generation_match= 0)
    sleep(3)


In [97]:
# Liste des clés pour alimenter le schéma d'importation dans BQ
name = 'rapports-visites/' +filenames[5]
blob = bucket.blob(name)
fileContent= (blob.download_as_string(client=None).decode())
json.loads(fileContent).keys()

dict_keys(['Tatouage', 'HoroDate', 'Agent', 'PC0', 'CM0', 'PC1', 'CM1', 'PC2', 'CM2', 'PC3', 'CM3', 'PC4', 'CM4', 'PC5', 'CM5', 'PC6', 'CM6', 'PC7', 'CM7', 'PC8', 'CM8', 'PC9', 'CM9', 'PC10', 'CM10', 'PC11', 'CM11', 'PC12', 'CM12', 'PC13', 'CM13', 'PC14', 'CM14'])

In [98]:
# Test de l'importation dans BQ pour la fonction logDepot
client  = bigquery.Client()
dataset  = client.dataset('rapports_visites')
table = dataset.table('LogDepot')

def format_schema(schema):
        formatted_schema = []
        for row in schema:
            formatted_schema.append(bigquery.SchemaField(row,'STRING', 'NULLABLE'))
        return formatted_schema
lst_schema_VBIS = ['Tatouage', 'HoroDate', 'Agent', 'PC0', 'CM0', 'PC1', 'CM1', 'PC2', 'CM2', 'PC3', 'CM3', 'PC4', 'CM4', 'PC5', 'CM5', 'PC6', 'CM6', 
                  'PC7', 'CM7', 'PC8', 'CM8', 'PC9', 'CM9', 'PC10', 'CM10', 'PC11', 'CM11', 'PC12', 'CM12', 'PC13', 'CM13', 'PC14', 'CM14']


job_config = bigquery.LoadJobConfig()
job_config.schema = format_schema(lst_schema_VBIS)
flJson=json.loads(fileContent)
stByt=','.join([flJson[k] for k in lst_schema_VBIS  ]).encode("utf-8")
job = client.load_table_from_file(io.BytesIO(stByt), table, job_config = job_config)